In [1]:
from __future__ import print_function

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

from pandas.stats.moments import ewma

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

%matplotlib inline

import datetime
dt = lambda: datetime.datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")

In [2]:
# Load data:
X = pd.read_csv('datasets/All_listings/sample_AMN_PCAed.csv', header = 0)
range_n_clusters = np.arange(2, int((34209-61)/2),2000)
#np.random.shuffle(range_n_clusters)
if_plot = False
results = []

In [3]:
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20
931833,68.100866,23.444897,21.902661,-0.683270,-7.588961,-4.839476,-7.458686,5.089244,-1.788304,3.120062,2.347572,1.649053,-4.112206,4.941023,4.417271,-3.611031,-1.323192,1.447385,5.154803,-1.038325
5359638,59.803677,-14.322003,-0.322916,15.543581,6.861194,-10.686353,5.870530,17.147783,-0.013002,-7.396568,7.258659,-0.624158,2.640960,-1.394403,2.388991,0.504676,1.928706,-2.160328,-0.659446,-2.094024
13327397,67.051546,19.054068,-12.291266,-11.067953,-4.848890,5.138821,-20.123026,-8.483664,-9.584005,10.890556,-12.231685,-3.201830,-6.940991,-9.563346,1.482292,-7.038796,1.631185,-0.950271,3.834554,-0.199780
726391,66.030401,25.930888,13.200808,-5.741762,10.170123,16.437917,7.785032,3.422430,9.740943,-1.537220,4.951095,5.077501,4.710491,4.511608,-14.532026,0.874322,-0.708795,1.959007,-5.635985,0.193720
14463911,69.916204,20.223802,-14.737813,4.049925,-4.703743,11.665783,-4.429664,8.141313,-0.447877,4.955242,-6.748633,7.022236,0.477127,4.716140,-5.619947,-3.542079,0.923375,-0.657936,3.480631,7.635465


In [4]:
X.shape

(34209, 20)

In [5]:

for n_clusters in range_n_clusters:
    print(dt(), "For n_clusters =", n_clusters,)
    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels, sample_size=1000)
    print("The average silhouette_score is :", silhouette_avg)
    results.append([n_clusters, silhouette_avg])
    if if_plot:
        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(X, cluster_labels)
        # Create a subplot with 1 row and 2 columns
        fig, ax1 = plt.subplots(1, 1)
        #fig.set_size_inches(18, 7)
    
        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 but in this example all
        # lie within [-0.1, 1]
        ax1.set_xlim([-0.1, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        y_lower = 10
        for i in range(n_clusters):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[cluster_labels == i]
    
            ith_cluster_silhouette_values.sort()
    
            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i
    
            color = cm.spectral(float(i) / n_clusters)
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)
    
            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    
            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples
    
        ax1.set_title(silhouette_avg)#"The silhouette plot for the various clusters.")
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")
    
        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
    
        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
    
        # 2nd Plot showing the actual clusters formed
    #    colors = cm.spectral(cluster_labels.astype(float) / n_clusters)
    #    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
    #                c=colors, edgecolor='k')
    
        # Labeling the clusters
    #    centers = clusterer.cluster_centers_
        # Draw white circles at cluster centers
    #    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
    #                c="white", alpha=1, s=200, edgecolor='k')
    
    #    for i, c in enumerate(centers):
    #        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
    #                    s=50, edgecolor='k')
    
    #    ax2.set_title("The visualization of the clustered data.")
    #    ax2.set_xlabel("Feature space for the 1st feature")
    #    ax2.set_ylabel("Feature space for the 2nd feature")
    #
        plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                      "with n_clusters = %d" % n_clusters),
                     fontsize=14, fontweight='bold')
    
        plt.show()

[2017-11-19 22:32:28] For n_clusters = 2
The average silhouette_score is : 0.15546872414
[2017-11-19 22:32:31] For n_clusters = 2002
The average silhouette_score is : 6.93474964992e-05
[2017-11-19 22:43:10] For n_clusters = 4002
The average silhouette_score is : 0.00768627194723
[2017-11-19 22:51:25] For n_clusters = 6002
The average silhouette_score is : 0.0115786497309
[2017-11-20 00:55:46] For n_clusters = 8002


MemoryError: 